<a href="https://colab.research.google.com/github/Luigib05/brain-mri-tumor-detection/blob/main/brain_MRI_images_brain_tumor_detection_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project name: **Brain MRI Images for Brain Tumor Detection**

This project uses a MRI images.jpg to train a computer vision models to detect presence or absence of brain tumor through tasnfer leaarning (TL). Interpretability with Grad-CAM will also be explored.

**1. Load data**

In [ ]:
from google.colab import files
files.upload()  # load data from local


**2. Unzip zip file**

In [ ]:
!unzip -q /content/archive.zip -d MRI_images/

**3. Import dependencies**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

**4. Data exploration**


In [ ]:
# Define base route
base_dir = '/content/MRI_images'

# Helper function to count images by class
def count_images(path):
    for folder in ['brain_tumor_dataset']:
        print(f"→ {folder.upper()}")
        for level in ['no', 'yes']:
            dir_class = os.path.join(path, folder, level)
            print(f"{level:<10}: {len(os.listdir(dir_class))} images")
        print()

# Function call
count_images('/content/MRI_images')

**4. Viewing examples**

In [ ]:
import matplotlib.image as mpimg
import random
import cv2  # useful for seeing image shape and type

# Paths
base_path = '/content/MRI_images/brain_tumor_dataset/'
levels = ['no', 'yes']

# Show 3 images per class
plt.figure(figsize=(12, 6))

for idx, level in enumerate(levels):
    path_class = os.path.join(base_path, level)
    images = os.listdir(path_class)
    samples = random.sample(images, 3)

    for j, nombre_img in enumerate(samples):
        img_path = os.path.join(path_class, nombre_img)
        img = mpimg.imread(img_path)

        plt.subplot(2, 3, idx*3 + j + 1)
        plt.imshow(img, cmap='gray')
        plt.title(f'{level}')
        plt.axis('off')

plt.tight_layout()
plt.show()


**5. Image shape and type**

In [ ]:
# Inspect a random image
img_path = os.path.join(base_path, 'no', random.choice(os.listdir(os.path.join(base_path, 'no'))))
img = cv2.imread(img_path)

print(f"Shape: {img.shape}")         # (height, width, channels)
print(f"Data type: {img.dtype}")     # ex. uint8


**6. Split the dataset into train, val and test**

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths
original_dir = '/content/MRI_images/brain_tumor_dataset'
output_base = '/content/MRI_images_split'
classes = ['yes', 'no']

# Create destination folders
for split in ['train', 'val', 'test']:
    for cls in classes:
        os.makedirs(os.path.join(output_base, split, cls), exist_ok=True)

# Separate by class
for cls in classes:
    src_dir = os.path.join(original_dir, cls)
    all_images = os.listdir(src_dir)

    train_imgs, temp_imgs = train_test_split(all_images, test_size=0.4, random_state=42)
    val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)

    for img in train_imgs:
        shutil.copy(os.path.join(src_dir, img), os.path.join(output_base, 'train', cls))
    for img in val_imgs:
        shutil.copy(os.path.join(src_dir, img), os.path.join(output_base, 'val', cls))
    for img in test_imgs:
        shutil.copy(os.path.join(src_dir, img), os.path.join(output_base, 'test', cls))


**6. Preparing the generators (ImageDataGenerator)**

a. Augmentation for train

b. Pixel normalization

c. Size definition (e.g., 224x224 for VGG)

In [ ]:
# Define the size of image for base model
image_size = (224, 224)
batch_size = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Validation and test: only normalization
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Directories
base_path = '/content/MRI_images_split'

# Generators
train_generator = train_datagen.flow_from_directory(
    os.path.join(base_path, 'train'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    os.path.join(base_path, 'val'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    os.path.join(base_path, 'test'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False  # for orderly evaluation
)


**7. Building the Model with Transfer Learning**

a. Load the base model (include_top=False)

b. Add your own dense layers

c. Freeze the initial layers

d. Compile

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# 1. Load VGG16 without the top layers
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# 2. Freeze all layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# 3. Adding our dense layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # binaria

# 4. Define the final model
model = Model(inputs=base_model.input, outputs=output)

# 5. Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 6. Summary
model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,780,481 (56.38 MB)

 Trainable params: 65,793 (257.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

**8. Model training**

to. Metrics: accuracy, loss, AUC

b. Callbacks: stopping early, checkpoint

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC

# 1. Recompile the model including the AUC metric
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc')]
)

# 2. Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='best_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

# 3. Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=callbacks
)



** 9. Load and evalulate the saved moldel (best_model.keras)**

In [ ]:
from tensorflow.keras.models import load_model

# Load the best trained model
model = load_model('best_model.keras')

# Evaluate in test
results = model.evaluate(test_generator)
print(f"\nTest loss: {results[0]:.4f}")
print(f"Test accuracy: {results[1]:.4f}")
print(f"Test AUC: {results[2]:.4f}")

10. Evaluation and visualization

a. Confusion matrix

b. ROC curve and AUC

c. Examples corrects vs. incorrects

**10.a. Confusion matrix**

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np

# Obtain predictions from the model
y_pred_prob = model.predict(test_generator)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# True labels
y_true = test_generator.classes

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Visualization
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['no', 'yes'], yticklabels=['no', 'yes'])
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Confusion matrix')
plt.show()

# Full report
print("\n" + classification_report(y_true, y_pred, target_names=['no', 'yes']))

** 10.b. ROC curve and AUC**

In [ ]:
from sklearn.metrics import roc_curve, auc

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


**10.c. Examples corrects vs. incorrects**

In [ ]:
# Obtain file names
filenames = test_generator.filenames

# Visualize examples of images with correct or incorrect diagnosis
import random

plt.figure(figsize=(12, 6))
for i in range(6):
    idx = random.randint(0, len(y_true)-1)
    true_label = y_true[idx]
    pred_label = y_pred[idx]
    color = 'green' if true_label == pred_label else 'red'

    img_path = os.path.join(test_generator.directory, filenames[idx])
    img = mpimg.imread(img_path)

    plt.subplot(2, 3, i+1)
    plt.imshow(img, cmap='gray')
    plt.title(f"Pred: {pred_label} | True: {true_label}", color=color)
    plt.axis('off')

plt.suptitle("Examples of test set (green = correct, red = error)")
plt.tight_layout()
plt.show()

**9. Interpretability with Grad-CAM**

a. Visualization: model activation onto image

b. Compare model activation between 'no' and 'yes'

In [ ]:
def generar_gradcam(idx, model, generator, class_names=['no', 'yes']):
    import tensorflow as tf
    import numpy as np
    import cv2
    import matplotlib.pyplot as plt
    from tensorflow.keras.preprocessing import image
    from tensorflow.keras.models import Model

    # Obtain path and image information
    img_path = os.path.join(generator.directory, generator.filenames[idx])
    true_label = y_true[idx]

    # Load and image preprocessing
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x /= 255.0

    # Predict
    pred_prob = model.predict(x)[0][0]
    pred_label = int(pred_prob > 0.5)


    # Identify tje last convolutional level
    last_conv_layer_name = None
    for layer in reversed(model.layers):
        if 'conv' in layer.name:
            last_conv_layer_name = layer.name
            break

    assert last_conv_layer_name is not None, "No se encontró una capa convolucional."

    # Intermediate model
    grad_model = Model(
        inputs=model.input,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Grad-CAM
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(x)
        loss = predictions[:, 0]  # asumiendo clase 1 = yes

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0].numpy()
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    # Superposition
    img_orig = cv2.imread(img_path)
    img_orig = cv2.resize(img_orig, (224, 224))
    heatmap = cv2.resize(heatmap, (img_orig.shape[1], img_orig.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img_orig, 0.6, heatmap_color, 0.4, 0)

    # Visualization
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(img_orig[..., ::-1])
    plt.title('Imagen Original')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(heatmap, cmap='jet')
    plt.title('Heatmap Grad-CAM')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(superimposed_img[..., ::-1])
    plt.title(f'Overlay\nTrue: {class_names[true_label]} | Pred: {class_names[pred_label]} ({pred_prob:.2f})')
    plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# Use a specific index
generar_gradcam(25, model, test_generator)

# See a picture with random pneumonia
random_idx = random.choice(np.where(y_true == 1)[0])
generar_gradcam(random_idx, model, test_generator)
